In [3]:
library(tidyverse)
library(dplyr)
library(ggplot2)
library(lubridate)
library(lmtest)
library(stargazer)
library(plm)
library(cowplot)

In [4]:
avocado = read.csv('avocado.csv')

In [5]:
tail(avocado)

,date,average_price,total_volume,X4046,X4225,X4770,total_bags,small_bags,large_bags,xlarge_bags,type,year,geography
30016,2020-05-17,1.16,51690121,15951219.7,9221698.7,728025.52,25788840.1,16896405.6,7972084.5,920350.00,conventional,2020,Total U.S.
30017,2020-05-17,1.58,2271254,150100.0,198457.0,5429.00,1917250.0,1121691.0,795559.0,0.00,organic,2020,Total U.S.
30018,2020-05-17,1.09,8667913,2081824.0,1020965.1,33410.85,5531562.9,2580802.5,2817078.8,133681.62,conventional,2020,West
30019,2020-05-17,1.71,384158,23455.0,39738.0,1034.00,319932.0,130051.0,189881.0,0.00,organic,2020,West
30020,2020-05-17,0.89,1240709,430203.1,126497.3,21104.42,662904.2,395909.3,265177.1,1817.81,conventional,2020,West Tex/New Mexico
30021,2020-05-17,1.58,36881,1147.0,1243.0,2645.00,31846.0,25621.0,6225.0,0.00,organic,2020,West Tex/New Mexico


## EDA

In [6]:
avocado$year = as.factor(avocado$year)
avocado$date = as.Date(avocado$date)
avocado$month  = factor(months(avocado$date), levels = month.name)

### 1. Avocado Price by Type

In [16]:
jpeg(file="price_by_type.jpeg", width=600, height=350)

options(repr.plot.width=6, repr.plot.height=3)

ggplot(avocado, aes(x=average_price, group=type, color=type)) + geom_density()+ theme_minimal() + 
theme(plot.title=element_text(hjust=0.5), legend.position="bottom") + labs(title="Avocado Price by Type")
dev.off()


pdf 
  2

In [8]:
seasonal_avo = avocado
seasonal_avo$month_year <- format(as.Date(avocado$date), "%Y-%m")
seasonal_avo$month <- format(as.Date(avocado$date), "%m")
seasonal_avo$year <- format(as.Date(avocado$date), "%Y")

### 2. Avocado Price by Month

In [15]:
jpeg(file="price_by_month.jpeg", width=600, height=350)

options(repr.plot.width=6, repr.plot.height=5)

conv_patterns <- seasonal_avo %>% select(month, average_price, type) %>% filter(type == "conventional") %>%
group_by(month) %>% summarize(avg=mean(average_price)) %>%
ggplot(aes(x=month, y=avg)) + geom_line(group=1, color="#7FB3D5") + 
labs(title="Conventional Avocados", x="Month", y="Average Price")

org_patterns <- seasonal_avo %>% select(month, average_price, type) %>% filter(type == "organic") %>%
group_by(month) %>% summarize(avg=mean(average_price)) %>%
ggplot(aes(x=month, y=avg)) + geom_line(group=1, color="#7FB3D5") + 
labs(title="Organic Avocados", x="Month", y="Average Price")

plot_grid(conv_patterns, org_patterns, nrow=2)

dev.off()

pdf 
  2

### 3. Avocado Price by Year

In [17]:
jpeg(file="price_by_year.jpeg", width=600, height=350)

options(repr.plot.width=6, repr.plot.height=5)

conv_patterns <- seasonal_avo %>% select(year, average_price, type) %>% filter(type == "conventional") %>%
group_by(year) %>% summarize(avg=mean(average_price)) %>%
ggplot(aes(x=year, y=avg)) + geom_line(group=1, color="#7FB3D5") + 
labs(title="Conventional Avocados", x="year", y="Average Price")

org_patterns <- seasonal_avo %>% select(year, average_price, type) %>% filter(type == "organic") %>%
group_by(year) %>% summarize(avg=mean(average_price)) %>%
ggplot(aes(x=year, y=avg)) + geom_line(group=1, color="#7FB3D5") + 
labs(title="Organic Avocados", x="year", y="Average Price")

plot_grid(conv_patterns, org_patterns, nrow=2)

dev.off()

pdf 
  2

### 4. Quantities by Month

In [18]:
jpeg(file="quantities_by_month.jpeg", width=600, height=350)

options(repr.plot.width=6, repr.plot.height=5)

conv_patterns_vol <- seasonal_avo %>% select(month, total_volume, type) %>% filter(type == "conventional") %>%
group_by(month) %>% summarize(avg=mean(total_volume)) %>%
ggplot(aes(x=month, y=avg)) + geom_line(group=1, color="#7FB3D5") + 
labs(title="Conventional Avocados", x="Month", y="Average Volume")

org_patterns_vol <- seasonal_avo %>% select(month, total_volume, type) %>% filter(type == "organic") %>%
group_by(month) %>% summarize(avg=mean(total_volume)) %>%
ggplot(aes(x=month, y=avg)) + geom_line(group=1, color="#7FB3D5") + 
labs(title="Organic Avocados", x="Month", y="Average Volume")

plot_grid(conv_patterns_vol, org_patterns_vol, nrow=2)
dev.off()

pdf 
  2

## Modeling

In [20]:
seasonal_avo = seasonal_avo %>% select('date', 'average_price', 'total_volume', 'type', 'year', 'geography',
                                     #'average_price_lag', 'average_price_lag_2', 
                                       'month_year', 'month')
head(seasonal_avo)

date,average_price,total_volume,type,year,geography,month_year,month
2015-01-04,1.22,40873.28,conventional,2015,Albany,2015-01,01
2015-01-04,1.79,1373.95,organic,2015,Albany,2015-01,01
2015-01-04,1.00,435021.49,conventional,2015,Atlanta,2015-01,01
2015-01-04,1.76,3846.69,organic,2015,Atlanta,2015-01,01
2015-01-04,1.08,788025.06,conventional,2015,Baltimore/Washington,2015-01,01
2015-01-04,1.29,19137.28,organic,2015,Baltimore/Washington,2015-01,01


In [21]:
grouped = seasonal_avo %>%
    group_by(type, month_year) %>% 
    summarise(average_price = mean(average_price),
              total_volume=mean(total_volume))  %>% 
    group_by(type) %>%
    mutate(average_price_lag = dplyr::lag(average_price, n = 1, default = NA)) %>%
    group_by(type) %>% 
    mutate(average_price_lag_2 = dplyr::lag(average_price_lag, n = 1, default = NA)) %>% ungroup() 
# head(grouped)

In [23]:
pdata = na.omit(pdata.frame(grouped, index = c("type", "month_year"), stringsAsFactors = FALSE))

In [24]:
model1 = lm(average_price ~ total_volume, pdata)
model2 = lm(average_price ~ total_volume + average_price_lag, pdata)
model3 = lm(average_price ~ total_volume + average_price_lag + average_price_lag_2, pdata)
model4 = plm(average_price ~ total_volume + average_price_lag+ average_price_lag_2, model='within', effect='time', pdata)
model5 = plm(average_price ~ total_volume + average_price_lag+ average_price_lag_2, model='within', effect='twoways', pdata)

In [26]:
#compare different lm without fixed effects
# stargazer(model1, model2, model3, type='text')

In [27]:
#compare the fixed effect models
# stargazer(model4, model5, type='text')

## 1 Check nonlinearity - no nonlinearity found

In [28]:
resettest(average_price ~ total_volume , power=2:11, type= c("fitted"), pdata)


	RESET test

data:  average_price ~ total_volume
RESET = 1.8188, df1 = 10, df2 = 114, p-value = 0.06489


In [29]:
resettest(average_price ~ total_volume +average_price_lag , power=2:3, type= c("fitted"), pdata)


	RESET test

data:  average_price ~ total_volume + average_price_lag
RESET = 0.28792, df1 = 2, df2 = 121, p-value = 0.7503


In [30]:
resettest(average_price ~total_volume +average_price_lag +average_price_lag_2, power=2:4, type= c("fitted"), pdata)


	RESET test

data:  average_price ~ total_volume + average_price_lag + average_price_lag_2
RESET = 0.60385, df1 = 3, df2 = 119, p-value = 0.6138


## 2. Test heteroskedasticity  

In [33]:
residuals = model5$residuals

In [34]:
pdata$residualis = residuals

In [36]:
bptest(model4)


	studentized Breusch-Pagan test

data:  model4
BP = 2.4988, df = 3, p-value = 0.4755


In [37]:
bptest(model5)


	studentized Breusch-Pagan test

data:  model5
BP = 2.4988, df = 3, p-value = 0.4755


In [39]:
coeftest(model4, vcov.=vcovHC, type='HC4')


t test of coefficients:

                       Estimate  Std. Error t value  Pr(>|t|)    
total_volume        -4.1042e-08  5.3189e-10 -77.162 < 2.2e-16 ***
average_price_lag    6.1596e-01  1.1421e-03 539.322 < 2.2e-16 ***
average_price_lag_2  2.2001e-01  2.5844e-04 851.317 < 2.2e-16 ***
---
Signif. codes:  0 ‘***’ 0.001 ‘**’ 0.01 ‘*’ 0.05 ‘.’ 0.1 ‘ ’ 1


In [40]:
coeftest(model5, vcov.=vcovHC, type='HC4')


t test of coefficients:

                       Estimate  Std. Error t value  Pr(>|t|)    
total_volume        -1.0743e-08  8.7979e-10 -12.211 < 2.2e-16 ***
average_price_lag    5.6825e-01  2.8222e-03 201.349 < 2.2e-16 ***
average_price_lag_2  1.4383e-01  6.9877e-03  20.584 < 2.2e-16 ***
---
Signif. codes:  0 ‘***’ 0.001 ‘**’ 0.01 ‘*’ 0.05 ‘.’ 0.1 ‘ ’ 1


## hypothesis testing on whether we should add fixed effects

In [41]:
pFtest(model4, model3)


	F test for time effects

data:  average_price ~ total_volume + average_price_lag + average_price_lag_2
F = 5.4004, df1 = 62, df2 = 60, p-value = 2.879e-10
alternative hypothesis: significant effects


In [42]:
pFtest(model5, model4)


	F test for twoways effects

data:  average_price ~ total_volume + average_price_lag + average_price_lag_2
F = 1.4445, df1 = 1, df2 = 59, p-value = 0.2342
alternative hypothesis: significant effects


In [52]:
stargazer(model1, model2, model3, model4, model5, model6,
          #se = list(NULL, NULL, NULL, robust_se_4, robust_se_5, robust_se_6),
          type = "html",
          out = "Term Proj.doc", 
          title = "Regression Results", 
          align = TRUE, 
          notes.append = FALSE, 
          notes = c(" *Significant at the 10% Level",
                    " **Significant at the 5% Level",
                    " ***Significant at the 1% Level"),
          omit.stat = c("f","adj.rsq","ser"), 
          add.lines = list(
                           c("Standard Errors"," "," "," ","Clustered","Clustered","Clustered"),
                           c("Type Dummies","No","No","No","No","Yes","Yes"),
                           c("Year-Month Dummies","No","No","No","Yes","No","Yes"),
                           c("RSS", 
                             round(sum(residuals(model1)^2), digits = 3),
                             round(sum(residuals(model2)^2), digits = 3),
                             round(sum(residuals(model3)^2), digits = 3),
                             round(sum(residuals(model4)^2), digits = 3),
                             round(sum(residuals(model5)^2), digits = 3),
                             round(sum(residuals(model6)^2), digits = 3))), 
          dep.var.labels = c("Average Price"),
          covariate.labels = c("Quantity", "Average price(t-1)", "Average price(t-2)"),
          column.labels = c("OLS", "OLS", "OLS", "FE", "FE", "FE"))


<table style="text-align:center"><caption><strong>Regression Results</strong></caption>
<tr><td colspan="7" style="border-bottom: 1px solid black"></td></tr><tr><td style="text-align:left"></td><td colspan="6"><em>Dependent variable:</em></td></tr>
<tr><td></td><td colspan="6" style="border-bottom: 1px solid black"></td></tr>
<tr><td style="text-align:left"></td><td colspan="6">Average Price</td></tr>
<tr><td style="text-align:left"></td><td colspan="3"><em>OLS</em></td><td colspan="3"><em>panel</em></td></tr>
<tr><td style="text-align:left"></td><td colspan="3"><em></em></td><td colspan="3"><em>linear</em></td></tr>
<tr><td style="text-align:left"></td><td>OLS</td><td>OLS</td><td>OLS</td><td>FE</td><td>FE</td><td>FE</td></tr>
<tr><td style="text-align:left"></td><td>(1)</td><td>(2)</td><td>(3)</td><td>(4)</td><td>(5)</td><td>(6)</td></tr>
<tr><td colspan="7" style="border-bottom: 1px solid black"></td></tr><tr><td style="text-align:left">Quantity</td><td>-0.00000<sup>***</sup></td><t